# Agenda
+ Some jupyter notebook basics
+ Julia
    - Written for scientific use
    - Familiar syntax to Matlab, R, Python
    - Speed of C
+ What is MXNet
    - Deep learning library
    - Built for scalable distributed computations
    - Supports many languages
    - Runs on CPU's, GPU's, smartphones, servers, clusters...
    - Uses both symbolic and imperative computations (think numpy)
+ GPU vs CPU computations


## Why Julia

<tr>
    <td> <img src="test.png" alt="Drawing" style="width: 600px;"/> </td>
    <td> <img src="benchmark.png" alt="Drawing" style="width: 600px;"/> </td>
    </tr>

## MXNet

![title](mnist.png)

In [ ]:
using MXNet

function get_mnist_providers(batch_size::Int; data_name=:data, label_name=:softmax_label, flat=true)
  filenames = mx.get_mnist_ubyte()

  train_provider = mx.MNISTProvider(image=filenames[:train_data],
                                    label=filenames[:train_label],
                                    data_name=data_name, label_name=label_name,
                                    batch_size=batch_size, shuffle=true, flat=flat, silent=true)
  eval_provider = mx.MNISTProvider(image=filenames[:test_data],
                                   label=filenames[:test_label],
                                   data_name=data_name, label_name=label_name,
                                   batch_size=batch_size, shuffle=false, flat=flat, silent=true)

  return (train_provider, eval_provider)
    end
;



In [ ]:
# data provider
batch_size = 5000
train_provider, eval_provider = get_mnist_providers(batch_size);

![title](nn.jpeg)

In [ ]:
# Define symbolic node for network architecture
data = mx.Variable(:data)

fc1  = mx.FullyConnected(data, name=:fc1, num_hidden=256)
act1 = mx.Activation(fc1, name=:relu1, act_type=:relu)
fc2  = mx.FullyConnected(act1, name=:fc2, num_hidden=128)
act2 = mx.Activation(fc2, name=:relu2, act_type=:relu)
fc6  = mx.FullyConnected(act2, name=:fc3, num_hidden=10)
mlp  = mx.SoftmaxOutput(fc6, name=:softmax)

# setup model. Here you can choose what hardware to use!
model = mx.FeedForward(mlp, context=[mx.gpu()]) 

# optimizer
optimizer = mx.SGD(lr=0.15, momentum=0.9, weight_decay=0.00001) 


## Training on CPU vs GPU

In [ ]:
# Change verbosity between 0-3 to see more or less iteration data

@time mx.fit(model, 
    optimizer, 
    train_provider,
    eval_data=eval_provider, 
        eval_metric = mx.MultiMetric([mx.Accuracy(), mx.MSE()]), 
    n_epoch=5, 
    verbosity=0, 
    callbacks=[mx.speedometer(frequency=100)])

In [ ]:
# Predict on an evaluation set. Each column respond to the probabilities of each number

mx.predict(model, eval_provider)

![title](gputech_f2.png)

## Check validation accuracy on trained model

In [ ]:
#--------------------------------------------------------------------------------
# Demonstration of the predict API
probs = mx.predict(model, eval_provider)

# collect all labels from eval data
labels = Array[]
for batch in eval_provider
  push!(labels, copy(mx.get(eval_provider, batch, :softmax_label)))
end
labels = cat(1, labels...)

# Now we use compute the accuracy
correct = 0
for i = 1:length(labels)
  # labels are 0...9
  if indmax(probs[:,i]) == labels[i]+1
    correct += 1
  end
end
println(mx.format("Accuracy on eval set: {1:.2f}%", 100correct/length(labels)))

In [ ]:
image1 = download("http://www.oceanographerschoice.com/log/wp-content/uploads/julia_benchmarks.png", "test.png");
image2 = download("https://www.ibm.com/developerworks/community/blogs/jfp/resource/BLOGS_UPLOADED_IMAGES/runtimes_205.png", "benchmark.png");
image3 = download("http://854320174.r.lightningbase-cdn.com/wp-content/uploads/2013/03/gputech_f2.png", "gpu.png")
image4 = download("http://cs231n.github.io/assets/nn1/neural_net.jpeg", "nn.jpeg")
image5 = download("https://www.researchgate.net/profile/Yuan_Tang3/publication/264203784/figure/fig3/AS:213858095505411@1427999151375/Samples-from-the-MNIST-database.png", "mnist.png")
;